In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
import sys

sys.path.append('/Users/kithmalgunathillake/desktop/crewAI/')

from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [5]:
sys.path.append('/Users/kithmalgunathillake/desktop/crewAI/Tailor Job Applications')
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/gkithmal',
    'personal_writeup': """Kithmal is an emerging Software Engineering Leader with hands-on experience in full-stack development, AI, and entrepreneurship. 
    As a co-founder of XENAI.IO, he has led the development of real-world tech solutions, including a facial recognition system for enterprise security. 
    He is proficient in Java, Python, and JavaScript, with strong knowledge of frameworks like Spring Boot, React, and FastAPI. 
    Kithmal is currently pursuing a B.Sc. in Computer Science with a specialization in Data Science, and actively explores deep learning and machine learning applications. 
    His entrepreneurial mindset, combined with technical depth and a drive for innovation, makes him an excellent fit for forward-thinking, impact-driven teams."""
}


In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/gkithmal) URLs, and personal write-up (Kithmal is an emerging Software Engineering Leader with hands-on experience in full-stack development, AI, and entrepreneurship. 
    As a co-founder of XENAI.IO, he has led the development of real-world tech solutions, including a facial recognition system for enterprise security. 
    He is proficient in Java, Python, and JavaScript, with strong knowledge of

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Kithmal

**Email:** kithmalg2005@gmail.com  
**Phone:** +94 77 123 4567

## Summary

Kithmal Gunathillake is an emerging Software Engineering Leader with hands-on experience in full-stack development, AI, and entrepreneurship. Proficient in Java, Python, and JavaScript, with expertise in frameworks like Spring Boot, React, and FastAPI. Currently pursuing a B.Sc. in Computer Science with a specialization in Data Science, actively exploring deep learning and machine learning applications. His entrepreneurial mindset, technical depth, and drive for innovation make him an excellent fit for forward-thinking, impact-driven teams.

## Work Experience

### XENAI.IO: Co-Founder & Engineering Lead (Maharagama, Sri Lanka) — 2024 - Present

- Co-founded XENAI.IO, a tech startup focused on AI-powered solutions and full-stack software development.
- Led the development of a facial recognition system for enterprise security.
- Managed a team in building scalable web applications using Java Spring Boot and React/Next.js.
- Successfully secured client projects and delivered MVPs aligned with business goals.

### Freelance Developer & AI Researcher — 2022 - 2024

- Worked on various client projects including full-stack development and machine learning research.
- Developed a FastAPI-based Disease Expert API using OpenAI models for intelligent insights on thoracic diseases.
- Collaborated with small businesses on digital product development, such as inventory management systems and analytics dashboards.

### University Projects & Research (Ongoing)

- Designed and implemented academic and side projects like a Ticket Management System using Java, MongoDB, and Angular.
- Actively exploring deep learning and neural networks through coursework and independent study.
- Passionate about applying academic knowledge to real-world business scenarios through startup ventures.

## Skills

- Proficient in Java, Python, and JavaScript
- Experienced with frameworks like Spring Boot, React, and FastAPI
- Strong knowledge of full-stack development and AI applications
- Deep understanding of data science and machine learning concepts
- Entrepreneurial mindset with a focus on innovation and problem-solving

## Education

**B.Sc. in Computer Science (Specialization in Data Science)**  
_Your University Name_ – _Expected Graduation: [Year]_

**Deep Learning & AI Specialization (In Progress)**  
Coursera, Fast.ai, and other platforms – _Independent Study_

**Entrepreneurship & Innovation**  
Self-Learning & Mentorship from Startup Founders – _2024–Present_

---

Kithmal Gunathillake is a rising talent in the software engineering and AI fields, bringing a unique blend of technical expertise and entrepreneurial spirit. His experience in leading engineering teams and developing innovative solutions makes him a valuable asset to any forward-thinking organization.

In [18]:
display(Markdown("./interview_materials.md"))

Interview Questions and Talking Points:

1. Can you share a specific project from your experience at XENAI.IO where you successfully implemented AI-powered solutions for enterprise security?
2. How do you approach team management and collaboration, especially in remote settings, based on your experience as a Co-Founder & Engineering Lead at XENAI.IO?
3. Tell us about a challenging client project you worked on as a Freelance Developer & AI Researcher and how you approached problem-solving to deliver results.
4. What motivated you to pursue a specialization in Data Science and how do you see it complementing your existing skills in Java, Python, and JavaScript?
5. Can you walk us through a technical project you worked on during your university studies that showcases your proficiency in full-stack development?
6. How do you stay updated on the latest trends and advancements in AI, machine learning, and deep learning, and how do you apply this knowledge to your work?
7. In what ways have you leveraged your entrepreneurial mindset and drive for innovation to create tangible impact in your projects and ventures?
8. Share an example of a successful collaboration with a small business on a digital product development project, highlighting your problem-solving and communication skills.
9. How do you see your experience in leading engineering teams and developing scalable web applications aligning with the goals and culture of our organization?
10. What sets you apart as a candidate for this role, and how do you envision leveraging your technical expertise and entrepreneurial spirit to contribute to our team's success?